# Weather Forcast - 5 day Minneapolis
---
- Makes API call to OpenWeatherMap to retrieve 5-day forcast
- outputs csv with data

In [3]:
import pandas as pd
import numpy as np
import requests

api_key = 'blocked'

In [4]:
# columns to get
weather_cols = ['date_time', 'clouds_all', 'temp_f', 'pressure', 'humidity', 'wind_speed', 'wind_deg']

# Empty dataframe
forcast_df = pd.DataFrame()

# Use column names defined above
forcast_df = forcast_df.reindex(columns =  weather_cols)
forcast_df

,date_time,clouds_all,temp_f,pressure,humidity,wind_speed,wind_deg


In [5]:
# base url for requests
base_url = "http://api.openweathermap.org/data/2.5/forecast?q="
units = "imperial"
city = 'minneapolis'

# Create query url + use it to get weather json for city
query_url = f"{base_url}{city}&units={units}&APPID={api_key}"
forcast_json = requests.get(query_url).json()

In [6]:
data = forcast_json['list']


# set up lists for column data
date_time = []
clouds_all = []
temp_f = []
pressure = []
humidity = []
wind_speed = []
wind_deg = []

# loop over all forcast jsons
for i in range(len(data)):
    
    date_time.append(data[i]['dt_txt'])
    clouds_all.append(data[i]['clouds']['all'])
    temp_f.append(data[i]['main']['temp'])
    pressure.append(data[i]['main']['pressure'])
    humidity.append(data[i]['main']['humidity'])
    wind_speed.append(data[i]['wind']['speed'])
    wind_deg.append(data[i]['wind']['deg'])


In [7]:
# set columns in df to the data retrieved from json
forcast_df['date_time'] = date_time
forcast_df['clouds_all'] = clouds_all
forcast_df['temp_f'] = temp_f
forcast_df['pressure'] = pressure
forcast_df['humidity'] = humidity
forcast_df['wind_speed'] = wind_speed
forcast_df['wind_deg'] = wind_deg

# set date_time col to type datetime
forcast_df['date_time'] = pd.to_datetime(forcast_df['date_time'])

forcast_df.head()

,date_time,clouds_all,temp_f,pressure,humidity,wind_speed,wind_deg
0,2019-11-18 18:00:00,79,39.25,1006,71,7.36,139
1,2019-11-18 21:00:00,11,40.71,1004,74,4.72,145
2,2019-11-19 00:00:00,22,36.81,1005,91,4.12,170
3,2019-11-19 03:00:00,94,35.92,1004,94,4.94,220
4,2019-11-19 06:00:00,97,38.32,1003,95,8.08,288


In [8]:
def expand_timestamps(df, ts_column):
    '''Function that accepts a dataframe and string for the timestamp column. It outputs
    lists for hour, day, month. Index must be continuous - gaps with throw off values.'''
    
    # empty lists for outputs
    hours, days, months = [], [], []
    
    # loop over length of timestamp columnm
    for i in range(len(df[ts_column])):
        
        # append minutes, hours, days, months
        hours.append(df[ts_column][i].hour)
        days.append(df[ts_column][i].dayofyear)
        months.append(df[ts_column][i].month)
        
    return hours, days, months

In [9]:
# add columns for hour, day, month
forcast_df['hour'], forcast_df['day_of_year'], forcast_df['month'] = \
                                                expand_timestamps(forcast_df, 'date_time')

In [10]:
def cyclical_encoding(time_data, time_type):
    '''Function to encode as cyclical a list or Pandas.Series of time elements. Accepts time_data
    and time_type ('hour', 'day_of_year', 'month') and returns list of sine and cosine coordinates
    for each time element as part of a unit circle.'''
    
    # set max_times by time type
    max_time = {
        'hour': 24,
        'month': 12,
        'day_of_year': 365
    }
    
    # compute sin and cos coordinates of unit circle for each element in time_data
    sin_time = np.sin(2 * np.pi * time_data / max_time[time_type])
    cos_time = np.cos(2 * np.pi * time_data / max_time[time_type])
    
    return sin_time, cos_time

In [11]:
# cyclical encoding for hour, day, month
forcast_df['sin_day'], forcast_df['cos_day'] = cyclical_encoding(forcast_df.day_of_year, 'day_of_year')
forcast_df['sin_hour'], forcast_df['cos_hour'] = cyclical_encoding(forcast_df.hour, 'hour')
forcast_df['sin_month'], forcast_df['cos_month'] = cyclical_encoding(forcast_df.month, 'month')

forcast_df.head()

,date_time,clouds_all,temp_f,pressure,humidity,wind_speed,wind_deg,hour,day_of_year,month,sin_day,cos_day,sin_hour,cos_hour,sin_month,cos_month
0,2019-11-18 18:00:00,79,39.25,1006,71,7.36,139,18,322,11,-0.674444,0.738326,-1.000000,-1.836970e-16,-0.5,0.866025
1,2019-11-18 21:00:00,11,40.71,1004,74,4.72,145,21,322,11,-0.674444,0.738326,-0.707107,7.071068e-01,-0.5,0.866025
2,2019-11-19 00:00:00,22,36.81,1005,91,4.12,170,0,323,11,-0.661635,0.749826,0.000000,1.000000e+00,-0.5,0.866025
3,2019-11-19 03:00:00,94,35.92,1004,94,4.94,220,3,323,11,-0.661635,0.749826,0.707107,7.071068e-01,-0.5,0.866025
4,2019-11-19 06:00:00,97,38.32,1003,95,8.08,288,6,323,11,-0.661635,0.749826,1.000000,6.123234e-17,-0.5,0.866025


In [12]:
# output to csv
forcast_df.to_csv('resources/weather_forcast_5day.csv')